## ASAS-SN SkyPatrol Python API Demo

The pyasassn client allows users to query the ASAS-SN input catalog and retrieve light curves from our database. These light curves are subject to live updates as we are running continuous photometry on our nightly images.



### Installation

Make sure your pip points to the appropriate Python >= 3.6 installation...
<pre><code>
    git clone https://github.com/asas-sn/skypatrol.git
    pip3 install skypatrol/ 
</code></pre>

### Tutorial

Create a SkyPatrolClient object. The client will automatically ping the server for the most recent catalog data.

In [2]:
from pyasassn.client import SkyPatrolClient

client = SkyPatrolClient()
client.catalogs

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Welcome to ASAS-SN Skypatrol!

Current Deployment Version: 0.6.17 (26 JAN 2024)
Please upgrade your client if not up to date.




Table Name:  stellar_main
Num Columns: 47
Num Targets: 98932961

Table Name:  master_list
Num Columns: 4
Num Targets: 109300848

Table Name:  asassn_discoveries
Num Columns: 11
Num Targets: 5716

Table Name:  comets
Num Columns: 1
Num Targets: 14815

Table Name:  swift
Num Columns: 56
Num Targets: 254936

Table Name:  allwiseagn
Num Columns: 15
Num Targets: 1354900

Table Name:  mdwarf
Num Columns: 32
Num Targets: 8927

Table Name:  glade_tmassx
Num Columns: 20
Num Targets: 1125414

Table Name:  glade_gwgc
Num Columns: 20
Num Targets: 52399

Table Name:  m_giants
Num Columns: 12
Num Targets: 4879809

Table Name:  glade_pgc
Num Columns: 20
Num Targets: 52400

Table Name:  glade_hyperleda
Num Columns: 20
Num Targets: 2459906

Table Name:  glade_sdssqso
Num Columns: 20
Num Targets: 297152

Table Name:  milliquas
Num Columns: 21
Num Targets: 1979676

Table Name:  fermi
Num Columns: 67
Num Targets: 5788

Table Name:  aavsovsx
Num Columns: 28
Num Targets: 1437528

Table Name:  morx
Num Colu

#### Main Catalog

The __stellar_main__ catalog contains the bulk of our targets. It was built off of ATLAS REFCAT2 and contains GAIA, TESS, SDSS, and ALLWISE identifiers where available.

In [ ]:
client.catalogs.stellar_main.head(12)

#### HEASARC Catalogs

The remaining catalogs were sourced from NASA's HEASARC archive. Each of these retains its original columnar data, though we have appended an __asas_sn_id__ for all of them.

In [ ]:
client.catalogs.aavsovsx.head(12)

#### The Master List

The __master_list__ contains __asas_sn_ids__ coordinates and catalog sources for all of our targets. All of our catalogs are cross-matched on the master list with a 2-arcsecond cone. 

In [ ]:
client.catalogs.master_list

### Cone Seach

Lets run a simple cone-search on the master list. 

In [ ]:
client.cone_search(ra_deg=270, dec_deg=88, radius=4, catalog='master_list')

### Random Curves 

For whatever reason, if you are interested in random targets from a given catalog, we can give you those too.

In [ ]:
client.random_sample(1000, catalog="aavsovsx")

### Query Lists

If you have a list of external identifiers you can query our catalogs using these. 
For the __stellar_main__ catalog, use the __id_col__ parameter.
For other catalogs you can search by name.

In [ ]:
my_tic_ids = [6658326, 46783395, 1021890]
client.query_list(my_tic_ids, catalog='stellar_main', id_col='tic_id', download=True)

In [ ]:
my_vso_id = 'ASASSN-V J182608.32-864925.1'
client.query_list(my_vso_id, catalog='aavsovsx', id_col='name')

### ADQL Queries

We have inculded a custom ADQL parser. That will allow users to query targets using this familiar SQL-like language. 
First, take note how we can use this to perform a cone-search.


In [ ]:
query = """
SELECT 
 * 
FROM stellar_main 
WHERE DISTANCE(ra_deg, dec_deg, 270, 88) <= ARCMIN(7.1)
"""
client.adql_query(query)

##### JOINS
Since we have cross matched all of our catalogs. We can use ADQL to explore targets accross catalogs.

In [ ]:
query = """
SELECT 
 asas_sn_id,
 chandra.name AS c_name,
 fermi.name 
FROM chandra 
JOIN fermi USING(asas_sn_id) 
"""
client.adql_query(query)

#### Complex Searches

Lets say we were searching for white dwarfs that crossmatched in the VSO catalog.

In [3]:
query = """
SELECT 
  asas_sn_id,
  gaia_id,
  pstarrs_g_mag,
  (gaia_mag - (5 * LOG10(plx) - 10)) AS g_mag_abs, 
  name 
FROM stellar_main 
JOIN aavsovsx USING(asas_sn_id)
WHERE 1=1
 AND pstarrs_g_mag < 5 
 AND (gaia_mag - (5 * LOG10(plx) - 10)) > 12
 AND (gaia_b_mag - gaia_r_mag) < 0.1 
"""
client.adql_query(query)

,asas_sn_id,gaia_id,pstarrs_g_mag,g_mag_abs,name
0,317828088771,5329693847558732288,4.680,14.128782,KX Vel
1,42950170136,414844568929565568,4.492,13.111701,NSV 260
2,446677174185,2900546759661734912,2.371,14.280309,NSV 2549
3,489626613110,3101768554465447936,4.569,12.836133,V0637 Mon
4,137439054844,482818271843093376,4.059,13.499305,NSV 16180
5,463856786747,5538726710386348160,4.779,12.286464,NSV 17607
6,309237713157,4240272953376907648,4.311,14.901798,eta Aql
7,592706543265,5589416086209311872,4.755,12.282585,NW Pup
8,377957140350,2934094096383830400,4.122,12.530326,iot CMa
9,240519370712,3324424744330466304,4.315,12.363800,NSV 16904


#### Downloading Curves

Any of the previous functions can take __mode='download_curves'__ as a parameter to download the lightcurves coresponding to these targets. 

The client will return a LightCurveCollection object which can be used for further analysis and plotting.

In [5]:
# Should take about 2-4 minutes

lcs2 = client.adql_query(query, download=True, threads=8)
lcs2.data

Pulled 57 of 57


,asas_sn_id,jd,flux,flux_err,mag,mag_err,limit,fwhm,image_id,camera,quality,phot_filter
0,94490650425,2.458581e+06,23357.940752,2.910325,5.478979,0.000135,13.492787,1.53,br149603,br,G,g
1,94490650425,2.459079e+06,65.102994,0.098103,11.866063,0.001638,17.173430,2.07,bB157137,bB,G,g
2,94490650425,2.458036e+06,27841.039447,3.999690,5.288352,0.000156,13.147575,1.41,br003459,br,G,g
3,94490650425,2.458644e+06,20619.801349,1.980601,5.614354,0.000104,13.910648,1.41,br164488,br,G,g
4,94490650425,2.458718e+06,67.719766,0.099305,11.823277,0.001594,17.160218,2.00,bB063835,bB,G,g
...,...,...,...,...,...,...,...,...,...,...,...,...
80544,231928250915,2.460437e+06,1683.701907,5.199004,8.334403,0.003356,12.862840,1.53,bj749519,bj,G,g
80545,231928250915,2.460424e+06,-8574.748362,6.871537,12.560006,99.999000,12.560006,2.09,bj745033,bj,B,g
80546,231928250915,2.460451e+06,-26.673822,5.511196,12.799526,99.999000,12.799526,1.55,bj754723,bj,G,g
80547,231928250915,2.460280e+06,270.181519,5.462580,10.320927,0.021976,12.809146,-99,bj693674,bj,None,g


In [ ]:
lcs2.stats()

#### Individual Curves

We can pull the individual curve from the LightCurveCollection object and plot.
Utilities are also available for lomb_scargle and period calculation.

In [ ]:
lightcurve = lcs[0]
lightcurve.meta

In [ ]:
lightcurve.plot()

In [ ]:
frequency, power, ls = lightcurve.lomb_scargle(plot=True)

In [ ]:
lightcurve.find_period(frequency, power, plot=True)

#### Saving

Finally, we can save the individual light curve or the entire collection to .csv

In [ ]:
# Individual
lightcurve.save(filename="vxmen.csv")
# Collection
lcs.save()